In [ ]:
!pip install kaleido
!pip install umap-learn
!pip install autokeras
!pip install fbpca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=d472fe4b8a1b222ca2442a2f0cda88724e5e61dd1a2e2c869f473073356f3db8
  Stored in directory: /root/.cache/pip/wheels/71/ae/f7/5b7e1e16a133d094431428169e9a8d9f750afe5459f0333c9b
Successfully built fbpca


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sklearn Libraries
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from fbpca import pca
import kaleido
import plotly.graph_objects as go
import plotly.express as px


# Tensorflow and Keras Libraries
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping

# Autokeras Library
import autokeras as ak

In [ ]:
!unzip /content/esm2_t36_3B_UR50D_inducedseq_embeddings.csv.zip
!unzip /content/best_model_fold_27.zip


Archive:  /content/esm2_t36_3B_UR50D_inducedseq_embeddings.csv.zip
  inflating: esm2_t36_3B_UR50D_inducedseq_embeddings.csv  
  inflating: __MACOSX/._esm2_t36_3B_UR50D_inducedseq_embeddings.csv  
Archive:  /content/best_model_fold_27.zip
   creating: best_model_fold_27/
  inflating: __MACOSX/._best_model_fold_27  
  inflating: best_model_fold_27/.DS_Store  
  inflating: __MACOSX/best_model_fold_27/._.DS_Store  
  inflating: best_model_fold_27/fingerprint.pb  
  inflating: __MACOSX/best_model_fold_27/._fingerprint.pb  
  inflating: best_model_fold_27/keras_metadata.pb  
  inflating: __MACOSX/best_model_fold_27/._keras_metadata.pb  
   creating: best_model_fold_27/variables/
  inflating: __MACOSX/best_model_fold_27/._variables  
  inflating: best_model_fold_27/saved_model.pb  
  inflating: __MACOSX/best_model_fold_27/._saved_model.pb  
   creating: best_model_fold_27/assets/
  inflating: __MACOSX/best_model_fold_27/._assets  
  inflating: best_model_fold_27/variables/variables.data-00000

In [ ]:
# read and process data
x = pd.read_csv("/content/esm2_t36_3B_UR50D_inducedseq_embeddings.csv")
scores = pd.read_csv("TM_Library_Expression_Retention.csv")
def is_string(val):
    return isinstance(val, str)
# Fix the deprecated applymap warning by using map instead
string_locations = x.map(is_string)
strings_in_df = x[string_locations].dropna(how='all').dropna(axis=1, how='all')
x.set_index('Unnamed: 0', inplace=True)
merged_df = scores.merge(x, left_on='Seq ID', right_index=True, how='inner')
merged_df['Average Retention Score'] = pd.to_numeric(merged_df['Average Retention Score'], errors='coerce')
merged_df['Average Induced Surface Score'] = pd.to_numeric(merged_df['Average Induced Surface Score'], errors='coerce')

# Get data
x = merged_df.iloc[:, 9:2569]
y = merged_df.iloc[:, 6]

n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold_num, (train_index, val_index) in enumerate(kf.split(x)):
    x_train, x_val = x.iloc[train_index], x.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Try a different approach - use TF autograph directly
    model_path = "/content/best_model_fold_27"

    # Use tf.saved_model.load instead to get direct access to the model
    imported_model = tf.saved_model.load(model_path)

    # Find the signature key for prediction
    signature_keys = list(imported_model.signatures.keys())
    prediction_function = imported_model.signatures[signature_keys[0]]

    # Convert pandas dataframe to tf tensor with double precision
    x_val_tensor = tf.convert_to_tensor(x_val.values, dtype=tf.float64)

    # Make prediction
    y_pred_tensor = prediction_function(x_val_tensor)

    # Extract the prediction values
    # Get the output tensor (usually the first item in the returned dictionary)
    output_key = list(y_pred_tensor.keys())[0]
    y_pred = y_pred_tensor[output_key].numpy()

    # Reshape if needed
    if len(y_pred.shape) > 1 and y_pred.shape[1] == 1:
        y_pred = y_pred.flatten()

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    print(r2_score)
    should_save = input("Save predictions? (y/n): ")
    if should_save.lower() == 'y':
      df = pd.DataFrame({'Actual': y_val, 'Predicted': y_pred})
      df.to_csv(f'DT_SFT_predictions_fold_{fold_num}.csv', index=False)

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(y_val, y_pred, alpha=0.6, edgecolors='w', linewidth=0.5, color='#1f77b4')
    ax.plot([np.min(y_val), np.max(y_val)], [np.min(y_val), np.max(y_val)], 'r', linestyle='--', color='gray')
    ax.set_title(f"Actual vs. Predicted", fontweight='bold', fontsize=15)
    ax.set_xlabel("Actual Values", fontsize=14)
    ax.set_ylabel("Predicted Values", fontsize=14)
    text_str = f'RMSE = {rmse:.2f}\nMAE = {mae:.2f}\nR^2 = {r2:.2f}'
    props = dict(boxstyle='round, pad=0.3', facecolor='whitesmoke', edgecolor='gray')
    ax.text(0.05, 0.95, text_str, transform=ax.transAxes, verticalalignment='top', bbox=props, fontsize=12)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='both', which='major', labelsize=12)
    plt.tight_layout()
    plt.savefig(f"predictions.png", dpi=1000)
    plt.show()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
!pip install leidenalg
!pip install python-igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.2 MB/s eta 0:00:00


In [ ]:
## 231003 Hank Jones
## Run Brian Hie's GeoSketch on library
## Set up packages in the PyCharm interpreter settings

from fbpca import pca
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import umap
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.express as px
from scipy.stats import pearsonr
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from scipy import stats
import community as community_louvain
import networkx as nx
import leidenalg as la
import igraph as ig

data = pd.read_csv("/content/esm2_t36_3B_UR50D_inducedseq_embeddings.csv")


## Find Strings in DF?
def is_string(val):
    return isinstance(val, str)

string_locations = data.applymap(is_string)
strings_in_df = data[string_locations].dropna(how='all').dropna(axis=1, how='all')
print(strings_in_df)

## Appears the strings are the row names
data.set_index('Unnamed: 0', inplace=True)
## Scale Data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
scaled_df = pd.DataFrame(data_scaled, index=data.index, columns=data.columns)

## Run PCA
U, s, Vt = pca(scaled_df.values, k=100) # E.g., 100 PCs.
data_dimred = U[:, :5] * s[:5]

## Compute the variance explained by each principle component
variance_explained = np.square(s) / np.sum(np.square(s))
proportion_variance_explained = variance_explained / np.sum(variance_explained)
grey_to_red_colorscale = [
    [0, '#D3D3D3'],
    [1, 'red']
]

## Create elbow plot
fig = go.Figure(data=[
    go.Scatter(y=proportion_variance_explained, mode='lines+markers',
               line=dict(color='black'), marker=dict(color='black'))
])

fig.update_layout(
    title='Proportion of Variance Explained by Principal Component',
    xaxis_title='Principal Component',
    yaxis_title='Proportion of Variance Explained',
    xaxis=dict(
        showgrid=False,  # Remove grid
        dtick=1,  # Show every principal component on x-axis
        linecolor='black',  # Set color to black
        linewidth=1  # Set line width
    ),
yaxis=dict(
        showgrid=False,  # Remove grid
        linecolor='black',  # Set color to black
        linewidth=1  # Set line width
    ),
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set paper background to white
    font=dict(
        family="Arial, sans-serif",  # Use a clear, sans-serif font
        size=12,
        color="black"
    )
)

fig.show()


## UMAP Plotting
## Project expression and retention scores onto graph
## Create a 2D UMAP embedding
reducer = umap.UMAP()
embedding = reducer.fit_transform(data_dimred)
scores = pd.read_csv("TM_Library_Expression_Retention.csv")
umap_df = pd.DataFrame(embedding, index=data.index, columns=['UMAP_1', 'UMAP_2'])
merged_df = scores.merge(umap_df, left_on='Seq ID', right_index=True, how='inner')
merged_df['Average Retention Score'] = pd.to_numeric(merged_df['Average Retention Score'], errors='coerce')
merged_df['Average Induced Surface Score'] = pd.to_numeric(merged_df['Average Induced Surface Score'], errors='coerce')

merged_df['Group'] = merged_df['Seq ID'].str.split('_').str[-1]

## Create the 2D scatter plot
fig = go.Figure(data=[
    go.Scatter(
        x=merged_df['UMAP_1'],
        y=merged_df['UMAP_2'],
        mode='markers',
        marker=dict(size=5, color=merged_df['Average Induced Surface Score'], colorscale=grey_to_red_colorscale, colorbar=dict(title='Average Induced Surface Score'), cmin=merged_df['Average Induced Surface Score'].min(),
    cmax=15)
    )
])

fig.update_layout(
    title='2D UMAP Projection with Overlayed Data',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig.show()

## Color by group

unique_groups = merged_df['Group'].unique()
group_mapping = {group: i for i, group in enumerate(unique_groups)}
mapped_groups = merged_df['Group'].map(group_mapping)

fig = go.Figure(data=[
    go.Scatter(
        x=merged_df['UMAP_1'],
        y=merged_df['UMAP_2'],
        mode='markers',
        marker=dict(
            size=5,
            color=mapped_groups,  # Use mapped numbers
            colorscale=px.colors.qualitative.Plotly,  # A qualitative colorscale
            colorbar=dict(tickvals=list(group_mapping.values()), ticktext=list(group_mapping.keys()), title='Group')
        )
    )
])


fig.update_layout(
    title='2D UMAP Projection with Overlayed Data',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig.show()


#########################################################################################################

## Subset data to group 1
## Run PCA
g1_data = scaled_df[data.index.str.contains(r'g1')]
U, s, Vt = pca(g1_data.values, k=100) # E.g., 100 PCs.
data_dimred = U[:, :5] * s[:5]

## Compute the variance explained by each principal component
variance_explained = np.square(s) / np.sum(np.square(s))
proportion_variance_explained = variance_explained / np.sum(variance_explained)


## Create elbow plot
fig = go.Figure(data=[
    go.Scatter(y=proportion_variance_explained, mode='lines+markers',
               line=dict(color='black'), marker=dict(color='black'))
])

fig.update_layout(
    title='Proportion of Variance Explained by Principal Component',
    xaxis_title='Principal Component',
    yaxis_title='Proportion of Variance Explained',
    xaxis=dict(
        showgrid=False,  # Remove grid
        dtick=1,  # Show every principal component on x-axis
        linecolor='black',  # Set color to black
        linewidth=1  # Set line width
    ),
yaxis=dict(
        showgrid=False,  # Remove grid
        linecolor='black',  # Set color to black
        linewidth=1  # Set line width
    ),
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set paper background to white
    font=dict(
        family="Arial, sans-serif",  # Use a clear, sans-serif font
        size=12,
        color="black"
    )
)

fig.show()

## UMAP Plotting
## Project expression and retention scores onto graph
## Create a 2D UMAP embedding
reducer = umap.UMAP()
embedding = reducer.fit_transform(data_dimred)
umap_df = pd.DataFrame(embedding, index=g1_data.index, columns=['UMAP_1', 'UMAP_2'])
merged_df = scores.merge(umap_df, left_on='Seq ID', right_index=True, how='inner')
merged_df['Average Retention Score'] = pd.to_numeric(merged_df['Average Retention Score'], errors='coerce')
merged_df['Average Induced Surface Score'] = pd.to_numeric(merged_df['Average Induced Surface Score'], errors='coerce')

## Create the 2D scatter plot
fig = go.Figure(data=[
    go.Scatter(
        x=merged_df['UMAP_1'],
        y=merged_df['UMAP_2'],
        mode='markers',
        marker=dict(size=5, color=merged_df['Average Induced Surface Score'], colorscale=grey_to_red_colorscale, colorbar=dict(title='Average Induced Surface Score'), cmin=merged_df['Average Induced Surface Score'].min(),
    cmax=15)
    )
])

fig.update_layout(
    title='2D UMAP Projection with Overlayed Data',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig.show()
fig.write_image("UMAP_g1.svg")


## Train Linear model
## Trim to surface scores >=0.5
train_data = data.merge(scores, left_index=True, right_on='Seq ID', how='inner')
trim_data = train_data[(train_data["Average Induced Surface Score"] >= 0.5)]
y = train_data["Average Induced Surface Score"]
col_to_drop = train_data.columns[-10:]
x = train_data.drop(columns=col_to_drop)

# Splitting the data into training and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

lm = LinearRegression()
lm.fit(x_train, y_train)
y_pred = lm.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
r2_value = lm.score(x_test, y_test)
corr_coef, _ = pearsonr(y_test, y_pred)

## Plot fit
plot_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
fig = px.scatter(plot_df, x='Actual', y='Predicted', title=f'Linear Regression on G1 ESM Embeddings (Pearson r: {corr_coef:.2f})')
fig.update_layout(shapes=[dict(type='line', yref="y", xref="x", y0=min(y_test), y1=max(y_test), x0=min(y_test), x1=max(y_test))])
fig.show()


##############################################################################################################
# # Setting up the Pois model
# poisson_model = sm.GLM(y_train, x_train, family=sm.families.Poisson()).fit()
# # Printing the summary
# print(poisson_model.summary())
# # Predicting on the test set
# y_pred_poisson = poisson_model.predict(x_test)
# corr_coef, _ = pearsonr(y_test, y_pred_poisson)
# # Data frame for plotting
# plot_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_poisson})

# fig = px.scatter(plot_df, x='Actual', y='Predicted',
#                  title=f'Poisson Regression (Pearson r: {corr_coef:.2f})')

# fig.add_trace(go.Scatter(x=plot_df['Actual'],
#                          y=plot_df['Predicted'],
#                          mode='markers',
#                          marker=dict(color='blue', opacity=0.5)))

# fig.add_shape(
#         dict(
#             type="line",
#             x0=min(y_test),
#             y0=min(y_test),
#             x1=max(y_test),
#             y1=max(y_test),
#             line=dict(color="black", width=2, dash="dash")
#         )
#     )

# fig.update_layout(
#     xaxis_title="Actual Values",
#     yaxis_title="Predicted Values",
#     font=dict(
#         family="Arial, sans-serif",
#         size=14,
#     ),
#     showlegend=False,
#     plot_bgcolor='white',
#     xaxis=dict(range=[0, 90]),  # set x-axis range
#     yaxis=dict(range=[0, 90])  # set y-axis range
# )

# fig.show()


# ##############################################################################################

# # Setting up the NB model
# neg_binom_model = sm.GLM(y_train, x_train, family=sm.families.NegativeBinomial()).fit()
# # Printing the summary
# print(neg_binom_model.summary())

# # Predicting on the test set
# y_pred_negbinom = neg_binom_model.predict(x_test)
# corr_coef, _ = pearsonr(y_test, y_pred_negbinom)
# # Data frame for plotting
# plot_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_negbinom})
# # Create the scatter plot
# fig = px.scatter(plot_df, x='Actual', y='Predicted',
#                  title=f'Negative Binomial Regression (Pearson r: {corr_coef:.2f})')

# fig.add_trace(go.Scatter(x=plot_df['Actual'],
#                          y=plot_df['Predicted'],
#                          mode='markers',
#                          marker=dict(color='blue', opacity=0.5)))

# fig.add_shape(
#         dict(
#             type="line",
#             x0=min(y_test),
#             y0=min(y_test),
#             x1=max(y_test),
#             y1=max(y_test),
#             line=dict(color="black", width=2, dash="dash")
#         )
#     )

# fig.update_layout(
#     xaxis_title="Actual Values",
#     yaxis_title="Predicted Values",
#     font=dict(
#         family="Arial, sans-serif",
#         size=14,
#     ),
#     showlegend=False,
#     plot_bgcolor='white',
#     xaxis=dict(range=[0, 60]),  # set x-axis range
#     yaxis=dict(range=[0, 60])  # set y-axis range
# )

# fig.show()
#################################################################################################
## Random Forests
## First 100 PCs
train_data = data.merge(scores, left_index=True, right_on='Seq ID', how='inner')
## trim_data = train_data[(train_data["Average Induced Surface Score"])]
y = train_data["Average Induced Surface Score"]
col_to_drop = train_data.columns[-10:]
x = train_data.drop(columns=col_to_drop)

# Splitting the data into training and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


pca = PCA(n_components=100)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)
rf = RandomForestRegressor(n_estimators=600, min_samples_split=15, min_samples_leaf=15, max_depth=10, n_jobs=-1)
rf.fit(x_train_pca, y_train)

# Predict on the test set
y_pred = rf.predict(x_test_pca)

# Compute the metrics
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
corr_coef, _ = pearsonr(y_test, y_pred)

# Create the plot
plot_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
fig = px.scatter(plot_df, x='Actual', y='Predicted',
                 title=f'Random Forest Regression 10000 Trees Default first 100 PCs (Pearson r: {corr_coef:.2f})')

fig.update_layout(shapes=[dict(type='line',
                               yref="y", xref="x",
                               y0=0,
                               y1=90,
                               x0=0,
                               x1=90)])

# Customize for a more minimalist design
fig.update_layout(
    showlegend=False,
    font=dict(
        family="Arial",
        size=12,
        color="Black"
    ),
    template="plotly_white",
    xaxis=dict(range=[0, 90]),  # set x-axis range
    yaxis=dict(range=[0, 90])   # set y-axis range
)

fig.show()


###############################################################################
## Evaluate Random Forest Algorithm

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def random_subset_train(data, scores, iterations=10):
    rmse_values = []
    prediction_records = []

    for fold in range(1, iterations + 1):
        # Random sample without replacement
        sample_data = data.sample(frac=0.8, replace=False, random_state=fold)
        train_data = sample_data.merge(scores, left_index=True, right_on='Seq ID', how='inner')

        y = train_data["Average Induced Surface Score"]
        col_to_drop = train_data.columns[-10:]
        x = train_data.drop(columns=col_to_drop)

        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=fold)

        pca = PCA(n_components=100)
        x_train_pca = pca.fit_transform(x_train)
        x_test_pca = pca.transform(x_test)

        rf = RandomForestRegressor(
            n_estimators=600,
            min_samples_split=15,
            min_samples_leaf=15,
            max_depth=10,
            n_jobs=-1,
            random_state=fold
        )
        rf.fit(x_train_pca, y_train)
        y_pred = rf.predict(x_test_pca)

        # Record RMSE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_values.append(rmse)

        # Store predictions vs actual values
        fold_df = pd.DataFrame({
            'fold': fold,
            'actual': y_test.values,
            'predicted': y_pred
        })
        prediction_records.append(fold_df)

    # Concatenate all folds' predictions
    all_predictions_df = pd.concat(prediction_records, ignore_index=True)

    return rmse_values, all_predictions_df


## Call function to assess how well optimized random forest performs
rmse_ranForest_vector, all_predictions_df = random_subset_train(data, scores, 10)
all_predictions_df.to_csv("/content/drive/MyDrive/SMPO/Figures/Figure2/rf_embeddings_line_plot.csv", index=False)

## Function to assess how much data we might need
def downsample_train(data, scores, fractions=[0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8], samples_per_fraction=20):
    results = []

    for frac in fractions:
        rmse_values = [frac]

        for _ in range(samples_per_fraction):
            # Random sample without replacement
            sample_data = data.sample(frac=frac, replace=False)
            train_data = sample_data.merge(scores, left_index=True, right_on='Seq ID', how='inner')

            y = train_data["Average Induced Surface Score"]
            col_to_drop = train_data.columns[-10:]
            x = train_data.drop(columns=col_to_drop)

            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

            pca = PCA(n_components=100)
            x_train_pca = pca.fit_transform(x_train)
            x_test_pca = pca.transform(x_test)

            rf = RandomForestRegressor(n_estimators=600, min_samples_split=15, min_samples_leaf=15, max_depth=10,
                                       n_jobs=-1)
            rf.fit(x_train_pca, y_train)
            y_pred = rf.predict(x_test_pca)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            rmse_values.append(rmse)

        results.append(rmse_values)
    # Convert to DataFrame
    df = pd.DataFrame(results)
    df.columns = ["Fraction"] + [f"Sample_{i+1}" for i in range(df.shape[1]-1)]
    return df


## Downsample and store the result
downSample_randForest_rmse_matrix = downsample_train(data, scores)

## Visualize Results
fraction_data = downSample_randForest_rmse_matrix.iloc[:, 0].values
performance_RMSE = downSample_randForest_rmse_matrix.iloc[:, 1:].values

mean_performance_RMSE = np.mean(performance_RMSE, axis=1)
sem = stats.sem(performance_RMSE, axis=1)  # Standard error of the mean
confidence_interval = sem * stats.t.ppf((1 + 0.95) / 2, len(performance_RMSE[0])-1)

fig = go.Figure()

## Add scatter points for each replicate
for col in range(performance_RMSE.shape[1]):
    fig.add_trace(go.Scatter(x=fraction_data, y=performance_RMSE[:, col], mode='markers', marker=dict(color='orange', opacity=0.5), showlegend=False))

# Add the mean response
fig.add_trace(go.Scatter(x=fraction_data, y=mean_performance_RMSE, mode='lines', line=dict(color='red'), name='Mean Performance'))

# Add the 95% confidence interval
fig.add_trace(go.Scatter(x=np.concatenate([fraction_data, fraction_data[::-1]]),
                         y=np.concatenate([mean_performance_RMSE - confidence_interval, (mean_performance_RMSE + confidence_interval)[::-1]]),
                         fill='toself', fillcolor='rgba(255,0,0,0.2)', line=dict(width=0),
                         name='95% Confidence Interval'))

fig.update_layout(
    title="Model Performance vs. Fraction Data Trained On",
    xaxis_title="Fraction of the Data",
    yaxis_title="RMSE",
    showlegend=True,
    template="plotly_white"
)

fig.show()


#####################################################################################
################## Linear Alignment Scores ##########################################
####################################################################################

alignmentData = pd.read_csv("Alignments.csv")


## Restructure, run PCA
alignmentData.set_index('Unnamed: 0', inplace=True)
U, s, Vt = pca(alignmentData.values, k=100) # E.g., 100 PCs.
data_dimred = U[:, :100] * s[:100]

## Compute the variance explained by each principal component
variance_explained = np.square(s) / np.sum(np.square(s))
proportion_variance_explained = variance_explained / np.sum(variance_explained)


## Create elbow plot
fig = go.Figure(data=[
    go.Scatter(y=proportion_variance_explained, mode='lines+markers',
               line=dict(color='black'), marker=dict(color='black'))
])

fig.update_layout(
    title='Proportion of Variance Explained by Principal Component',
    xaxis_title='Principal Component',
    yaxis_title='Proportion of Variance Explained',
    xaxis=dict(
        showgrid=False,  # Remove grid
        dtick=1,  # Show every principal component on x-axis
        linecolor='black',  # Set color to black
        linewidth=1  # Set line width
    ),
yaxis=dict(
        showgrid=False,  # Remove grid
        linecolor='black',  # Set color to black
        linewidth=1  # Set line width
    ),
    plot_bgcolor='white',  # Set plot background to white
    paper_bgcolor='white',  # Set paper background to white
    font=dict(
        family="Arial, sans-serif",  # Use a clear, sans-serif font
        size=12,
        color="black"
    )
)

fig.show()


## UMAP Plotting
## Project expression and retention scores onto graph
## Create a 2D UMAP embedding
data_dimred = U[:, :5] * s[:5]
reducer = umap.UMAP()
embedding = reducer.fit_transform(data_dimred)
## Add Names to umap_df
umap_df = pd.DataFrame(embedding, index=scores.loc[1:1923, "Seq ID"], columns=['UMAP_1', 'UMAP_2'])
merged_df = scores.merge(umap_df, left_on='Seq ID', right_index=True, how='inner')
merged_df['Average Retention Score'] = pd.to_numeric(merged_df['Average Retention Score'], errors='coerce')
merged_df['Average Induced Surface Score'] = pd.to_numeric(merged_df['Average Induced Surface Score'], errors='coerce')


grey_to_red_colorscale = [
    [0, '#D3D3D3'],
    [1, 'red']
]

## Create the 2D scatter plot
fig = go.Figure(data=[
    go.Scatter(
        x=merged_df['UMAP_1'],
        y=merged_df['UMAP_2'],
        mode='markers',
        marker=dict(size=5, color=merged_df['Average Induced Surface Score'], colorscale= grey_to_red_colorscale, colorbar=dict(title='Average Induced Surface Score'), cmin=merged_df['Average Induced Surface Score'].min(),
    cmax=5)
    )
])

fig.update_layout(
    title='2D UMAP Projection with Overlayed Data',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig.show()




###############################################################
# 231130 HJ modify umap plots from ESM

embeddings = pd.read_csv("esm2_t36_3B_UR50D_inducedseq_embeddings.csv")

## Find Strings in DF?
def is_string(val):
    return isinstance(val, str)

string_locations = embeddings.applymap(is_string)
strings_in_df = embeddings[string_locations].dropna(how='all').dropna(axis=1, how='all')
print(strings_in_df)

## Appears the strings are the row names
embeddings.set_index('Unnamed: 0', inplace=True)
## Scale Data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(embeddings)
scaled_df = pd.DataFrame(data_scaled, index=embeddings.index, columns=embeddings.columns)

## Run PCA on embeddings
U, s, Vt = pca(scaled_df.values, k=100) # E.g., 100 PCs.
data_dimred = U[:, :5] * s[:5]
reducer = umap.UMAP()
embedding = reducer.fit_transform(data_dimred)
umap_df = pd.DataFrame(embedding, index=embeddings.index, columns=['UMAP_1', 'UMAP_2'])

## Merge predictions with embeddings
merged_df = scores.merge(umap_df, left_on='Seq ID', right_index=True, how='inner')
merged_df['Average Retention Score'] = pd.to_numeric(merged_df['Average Retention Score'], errors='coerce')
merged_df['Average Induced Surface Score'] = pd.to_numeric(merged_df['Average Induced Surface Score'], errors='coerce')
merged_df['Group'] = merged_df['Seq ID'].str.split('_').str[-1]
filtered_df = merged_df[merged_df['Group'].isin(['g1', 'g2', 'g3', 'g4'])]

def create_umap_plot(df, color_column=None, cmin=None, cmax=15):
    fig = go.Figure()  # Initialize the figure at the start

    if color_column is not None:
        if pd.api.types.is_categorical_dtype(df[color_column]) or df[color_column].dtype == 'object':
            # Create a plot for each category
            unique_categories = df[color_column].unique()
            colors = px.colors.qualitative.Plotly
            for i, category in enumerate(unique_categories):
                category_df = df[df[color_column] == category]
                fig.add_trace(go.Scatter(
                    x=category_df['UMAP_1'],
                    y=category_df['UMAP_2'],
                    mode='markers',
                    marker=dict(size=5, color=colors[i % len(colors)]),
                    name=category  # This will be used for the legend entry
                ))
        else:
            # For continuous data
            grey_to_red_colorscale = [
                [0, '#D3D3D3'],  # Light grey
                [1, 'red']
            ]
            fig.add_trace(go.Scatter(
                x=df['UMAP_1'],
                y=df['UMAP_2'],
                mode='markers',
                marker=dict(
                    size=5,
                    color=df[color_column],
                    colorscale=grey_to_red_colorscale,
                    colorbar=dict(title=color_column),
                    cmin=cmin if cmin is not None else df[color_column].min(),
                    cmax=cmax
                )
            ))
    else:
        # Default plot when no color column is specified
        fig.add_trace(go.Scatter(
            x=df['UMAP_1'],
            y=df['UMAP_2'],
            mode='markers',
            marker=dict(size=5)
        ))

    fig.update_layout(
        title='2D UMAP Projection with Overlayed Data',
        xaxis_title='UMAP Dimension 1',
        yaxis_title='UMAP Dimension 2',
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Arial, sans-serif", size=12, color="black")
    )

    fig.show()

create_umap_plot(filtered_df, color_column='Group')


# Cluster on g1 and g2, randomly sample TMDs for JXTA tests

embeddings = pd.read_csv("esm2_t36_3B_UR50D_inducedseq_embeddings.csv")


## Find Strings in DF?
def is_string(val):
    return isinstance(val, str)

string_locations = embeddings.applymap(is_string)
strings_in_df = embeddings[string_locations].dropna(how='all').dropna(axis=1, how='all')
print(strings_in_df)

## Appears the strings are the row names
embeddings.set_index('Unnamed: 0', inplace=True)
## Scale Data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(embeddings)
scaled_df = pd.DataFrame(data_scaled, index=embeddings.index, columns=embeddings.columns)

## Run PCA on embeddings
U, s, Vt = pca(scaled_df.values, k=100) # E.g., 100 PCs.
data_dimred = pd.DataFrame(U[:,:20] * s[:20], index=embeddings.index)


# Merge, filter for G1 & G2
merged_df = scores.merge(data_dimred, left_on='Seq ID', right_index=True, how='inner')
merged_df['Group'] = merged_df['Seq ID'].str.split('_').str[-1]
filtered_df = merged_df[merged_df['Group'].isin(['g1', 'g2'])]
data_dimred = filtered_df.iloc[:, 8:28]
data_dimred.index = filtered_df.iloc[:, 0]

# Make UMAP
reducer = umap.UMAP()
embedding = reducer.fit_transform(data_dimred)
umap_df = pd.DataFrame(embedding, index=data_dimred.index, columns=['UMAP_1', 'UMAP_2'])

# Create a NetworkX graph from PCA-reduced data
G = ig.Graph()
G.add_vertices(len(data_dimred))
for i in range(len(data_dimred)):
    for j in range(len(data_dimred)):
        weight = np.linalg.norm(data_dimred[i] - data_dimred[j])
        G.add_edge(i, j, weight=weight)

# Finding the best partition with the Leiden algorithm
partition = la.find_partition(G, la.ModularityVertexPartition, weights="weight")
partition_dict = {node: membership for membership, community in enumerate(partition) for node in community}

# Add the cluster labels to the UMAP DataFrame
umap_df['Cluster'] = pd.Series(partition_dict)

# Visualize the Clusters in UMAP Space
fig = go.Figure()

for cluster_id in set(partition_dict.values()):
    cluster_data = umap_df[umap_df['Cluster'] == cluster_id]
    fig.add_trace(go.Scatter(
        x=cluster_data['UMAP_1'],
        y=cluster_data['UMAP_2'],
        mode='markers',
        marker=dict(size=5),
        name=f'Cluster {cluster_id}'
    ))

fig.update_layout(
    title='UMAP Projection with Leiden Clusters',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig.show()





<ipython-input-7-bdbd0ee097b8>:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_locations = data.applymap(is_string)


             Unnamed: 0
0     Q9H7M9_195-215_g1
1     Q9BZD6_114-134_g1
2     Q96H15_315-335_g1
3     P35916_776-796_g1
4     P22309_491-507_g1
...                 ...
5619        EEK_003_g11
5620        EEK_004_g11
5621        EEK_005_g11
5622        EEK_006_g11
5623        EEK_007_g11

[5624 rows x 1 columns]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



Mean Squared Error: 157.9457968106914


Mean Squared Error: 57.946642159715424


KeyboardInterrupt: 